# Logistic Regression implementation

For this exercise, we will perform Logistic Regression. Logistic regression is appropriate since we are predicting a categorical target outcome given a feature set, $X$. 

Following ISL, section 4.3, we can express the logistic function for the log-odds ratio or logit for the multiple predictor case as:

$\begin{equation}log\left(\frac{p(X)}{1-p(X)}\right) = \beta_{0} + \beta_{1} X_{1} +\ldots + \beta_{p} X_{p}\end{equation}$

where $X=(X_{1},\ldots,X_{p})$ are $p$ predictors.

Therefore, the logit regression model is linear in $X$. The logistic function can be written as the logit function, which expresses the underlying sigmoid function:

$\begin{equation}p(X)=\frac{exp^{\beta_{0} + \beta_{1}X_{1} + \ldots +\beta_{p}X_{p}}}{1+ exp^{\beta_{0} + \beta_{1}X_{1} + \ldots +\beta_{p}X_{p}}}\end{equation}$

or, alternatively, as 

$\begin{equation}logistic(\boldsymbol{w}^T\cdot\mathbf{X}) = logit^{-1}(\boldsymbol{w}^T\cdot\mathbf{X}) = \frac{1}{1+exp^{-\boldsymbol{w}^T\cdot\mathbf{X}'_i}}\end{equation}$

where  $\boldsymbol{w}$  denotes the model parameters and $b$ represents the bias.

The Logistic Regression loss function can be described mathematically as:

$\begin{equation}
f(\boldsymbol{w}) = \sum_{i=1}^{n}log(1+exp^{-y(\boldsymbol{w}^T\cdot\mathbf{x}'_i + b)})\end{equation}$


Or, loss (L)<br>

$\begin{equation}L = \frac{1}{2N} \sum_{i}(y_{i} - a_{i})^2\end{equation}$
<br>

Using chain rule we can reduce this to : <br>

$\begin{equation}\frac{dL}{dW} = \frac{1}{N} \sum_{i}-(y_{i} - a_{i}).a_{i}(1-a_{i}).x_{i}\end{equation}$


#### Evaluation criteria

Log loss is used to evaluate logistic regression and tree-based algorithms. It is the negative log-likelihood of the Bernoulli model. As mentioned in the reference paper, Aryafar, et al. (2017), log loss is often used as an indicator for model performance in online advertising. Minimizing log loss indicates that the conditional probability of clicking on an ad given a query by the user should converge to the expected click rate. Here the conditional probability can be written an $P(c|q,l)$, where $c$ is the binary response of clicking on an advertisement or not, $q$ is the indicator for a query issued by a user and $l$ is a given advertisement listing. A model with lower log loss is preferred. This is referred to in the reference paper Aryafar, et al. (2017).

The following logarithmic loss function is used to measure the evaluate the model proposed:

$\begin{equation}
\boldsymbol{logloss} = -\frac{1}{L}\sum_{i=1}^{L}y_{i}log p_{i}+(1-y_{i})log(1-p_{i})\end{equation}$

where $L$ is the number of instances, $y_{i} \in {0,1}$ is the label of the $i^{th}$ instance, and $p_{i}$ is the probability of that the $i^{th}$ instance is clicked. 

As far as confusion matrix metrics, precision and recall are used.

Following the example guidance for implementation in MLLib in:

https://github.com/MingChen0919/learning-apache-spark/blob/master/notebooks/06-machine-learning/classification/logistic-regression.ipynb

## Logistic Regression - Model Training and Evaluation

In [5]:
# To run, execute in bash shell
# python submit_job_to_bigger_cluster.py --project_id=${PROJECT_ID} --zone=central1-a --cluster_name=w261finalproject --gcs_bucket=${BUCKET_NAME} --key_file=$HOME/MIDS/w261.json --create_new_cluster --pyspark_file=data_processing.py

In [10]:
# load and import libaries
import time
import math

from pyspark.sql import SQLContext
from pyspark.sql import SparkSession 


from pyspark.sql.types import *
from pyspark.sql.functions import lit, when, col, approx_count_distinct, mean, log, udf
from pyspark.ml.feature import FeatureHasher

from pyspark.ml.classification import LogisticRegression

from pyspark.conf import SparkConf


app_name = "logistic_regression"

#set spark session options
conf=SparkConf()
conf.set("spark.sql.parquet.compression.codec", "snappy")
#initiate the spark session
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

#print all the session options and settings
for object in sc.getConf().getAll():
    print(object)

('spark.driver.port', '36825')
('spark.rdd.compress', 'True')
('spark.serializer.objectStreamReset', '100')
('spark.master', 'local[*]')
('spark.executor.id', 'driver')
('spark.submit.deployMode', 'client')
('spark.app.id', 'local-1576351808175')
('spark.app.name', 'logistic_regression')
('spark.ui.showConsoleProgress', 'true')
('spark.driver.host', 'docker.w261')


In [4]:
spark

Assign global variables and input and output files

In [6]:
#set the global variables
MAX_ITER = 10       #number of model training epochs
THRESHOLD = 0.45    #model training threshold for classification probability
EPSILON = 1e-16     #variable to force bounded solution for log transforms

#import one hot encoded file
#INPUT_FILE = 'gs://261_projectdata/261project_data/df_ohe_10k.parquet'
INPUT_FILE = 'data/df_ohe_300.parquet'

# OUTPUT_FILE = 'gs://261_projectdata/results/ohe_10k_result.csv'
OUTPUT_FILE = 'data/ohe_result.csv'

Read in parquet file into a Spark dataframe

In [7]:
df_pq = spark.read.parquet(INPUT_FILE)\
                .select('y', 'features')

print(f' Total number of Rows = {df_pq.count()}')

 Total number of Rows = 46048


In [8]:
#remove records where the label is missing
df_pq = df_pq.na.drop(subset = ["y"])

print(f' Dropped NULL values from prediction variable. Total number of Rows = {df_pq.count()}')
#create train-test split
train_y_all, test_y_all = df_pq.randomSplit([0.8, 0.2])
df_pq.unpersist()

#change the name of the dependent variable column
test = test_y_all.selectExpr("y as labels", "features")
print(f' Total number of Rows in the TEST set = {test.count()}')
test.show(n=5)
#force a binary classification 0 or 1 on the training data and save as column 'labels'
train_y = train_y_all.withColumn('labels', when(col('y') > 0.0, lit(1)).otherwise(lit(0)))
train_y_all.unpersist()
#remove the old dependent variable
train = train_y.drop('y')
train_y.unpersist()

print(f' Total number of Rows in the TRAIN set = {train.count()}')

train.show(n=10)


#Get the percentage of values in the positive class, to set the baseline log loss
mean_prob = train.select(mean(col('labels'))).collect()[0]['avg(labels)']
print(f'Positive class in the training data = {mean_prob * 100} %')

#assign mean probability as the baseline probability estimate
train_prob = train.withColumn('base_prob', lit(mean_prob))
train.unpersist()

# calculate baseline logloss  
train_ll = train_prob.withColumn('logloss', when(col('labels') > 0.0, - log(col('base_prob') + EPSILON))\
                                       .otherwise( - log(1.0 - col('base_prob') + EPSILON)))
train_prob.unpersist()
#get the mean of the logloss
mean_ll = train_ll.select(mean(col('logloss'))).collect()[0]['avg(logloss)']
print(f'Baseline Log loss for the training data = {mean_ll}')

#specify the model and hyperparameters
lr = LogisticRegression(featuresCol = 'features',
                        labelCol='labels', 
                        maxIter = MAX_ITER, 
                        standardization = False,
                        elasticNetParam = 0.0, 
                        threshold = THRESHOLD
                        )

start = time.time()
print('Training model...')
lr_model = lr.fit(train_ll)
time_taken = time.time() - start
print(f'Training completed ...in {time_taken}')



 Dropped NULL values from prediction variable. Total number of Rows = 46048
 Total number of Rows in the TEST set = 9282
+------+--------------------+
|labels|            features|
+------+--------------------+
|     0|(545,[0,11,29,52,...|
|     0|(545,[0,11,29,52,...|
|     0|(545,[0,11,29,52,...|
|     0|(545,[0,11,29,52,...|
|     0|(545,[0,11,29,52,...|
+------+--------------------+
only showing top 5 rows

 Total number of Rows in the TRAIN set = 36766
+--------------------+------+
|            features|labels|
+--------------------+------+
|(545,[0,11,29,52,...|     0|
|(545,[0,11,29,52,...|     0|
|(545,[0,11,29,52,...|     0|
|(545,[0,11,29,52,...|     0|
|(545,[0,11,29,52,...|     0|
|(545,[0,11,29,52,...|     0|
|(545,[0,11,29,52,...|     0|
|(545,[0,11,29,52,...|     0|
|(545,[0,11,29,52,...|     0|
|(545,[0,11,29,52,...|     0|
+--------------------+------+
only showing top 10 rows

Positive class in the training data = 25.444704346407004 %
Baseline Log loss for the traini

Define user defined functions for sigmoid function and extracting vector; then extract coefficients and confusion matrix metrics (precision and recall)

In [9]:
def sigmoid(rawPrediction):
    #apply affine transformation
    return 1 / (1 + math.exp(- rawPrediction))

def extract_from_vector(vec, i):
    """ Input: VectorUDT data type
        Output: float type at index (i)
    """
    try:
        return float(vec[i])
    except ValueError:
        return None

#register UDFs    
get_probability = udf(sigmoid, DoubleType())
get_val = udf(extract_from_vector, DoubleType())

#Get predictions on test data
result = lr_model.transform(test)

#Apply affine transformation to the linear regression output
result2 = result.withColumn('calc_prob', get_probability(get_val("rawPrediction", lit(1))))
# calculate log loss
result3 = result2.withColumn('logloss', when(col('labels') > 0.0, - log(col('calc_prob') + EPSILON))\
                                       .otherwise( - log(1.0 - col('calc_prob') + EPSILON)))


test_mean_ll = result3.select(mean(col('logloss'))).collect()[0]['avg(logloss)']
print(f'Baseline Log loss for the training data = {mean_ll}')
print(f'MODEL Log loss for the training data = {test_mean_ll}')

result3.select(['features', 'labels' ,'prediction', 'calc_prob', 'probability', 'logloss']).show(n=10, truncate=True)

analyze = result3.select('labels', 'prediction')
analyze = analyze.withColumn("prediction", analyze["prediction"].cast(IntegerType()))
analyze = analyze.withColumn("labels", analyze["labels"].cast(IntegerType()))
result3.unpersist()
#Create SQL queries table
sqlContext.registerDataFrameAsTable(analyze, "results")
#Get metrics for precision recall calculations
TP = spark.sql("""SELECT COUNT(prediction)
                    FROM results
                    WHERE labels = 1 AND prediction = 1
                    """).collect()[0][0]

TN = spark.sql("""SELECT COUNT(prediction)
                    FROM results
                    WHERE labels = 0 AND prediction = 0
                    """).collect()[0][0]

FP = spark.sql("""SELECT COUNT(prediction)
                    FROM results
                    WHERE labels = 0 AND prediction = 1
                    """).collect()[0][0]

FN = spark.sql("""SELECT COUNT(prediction)
                    FROM results
                    WHERE labels = 1 AND prediction = 0
                    """).collect()[0][0]


print(f'Precision = {TP/(TP + FP) * 100} %')

print(f'Recall = {TP/(TP + FN) * 100} %')

features = [x["name"] for x in sorted(train_ll.schema["features"].metadata["ml_attr"]["attrs"]["binary"], 
                                      key=lambda x: x["idx"]
                                     )]

#Extract feature names and indices from table metadata
schema = StructType([StructField("feature", StringType()),
                    StructField("coeff", FloatType())
                    ])

# Create dataframe of coefficients
result_df = spark.createDataFrame(zip(features, lr_model.coefficients.tolist()), schema=schema)
print(result_df.show(n=5))

#save to CSV file
result_df.coalesce(1).write.csv(OUTPUT_FILE, mode='overwrite')

Baseline Log loss for the training data = 0.5671681910174072
MODEL Log loss for the training data = 0.5012941120993041
+--------------------+------+----------+-------------------+--------------------+-------------------+
|            features|labels|prediction|          calc_prob|         probability|            logloss|
+--------------------+------+----------+-------------------+--------------------+-------------------+
|(545,[0,11,29,52,...|     0|       0.0| 0.2454877025064577|[0.75451229749354...| 0.2816837020421912|
|(545,[0,11,29,52,...|     0|       0.0|0.11046406477582439|[0.88953593522417...|0.11705537333963632|
|(545,[0,11,29,52,...|     0|       0.0| 0.0666469095631895|[0.93335309043681...|0.06897170338584309|
|(545,[0,11,29,52,...|     0|       0.0|0.32543339638649177|[0.67456660361350...|0.39368486304473055|
|(545,[0,11,29,52,...|     0|       0.0| 0.3880836650158173|[0.61191633498418...| 0.4911597133573236|
|(545,[0,11,29,52,...|     0|       0.0| 0.2028108255449076|[0.79

## Logistic Regression - Hashing Function - Training and Evaluation

Assign global variables

In [11]:
#set the global variables
NUM_FEATURES = 2**18  #parameter for setting the number of mapped feature space
MAX_ITER = 10       #number of model training epochs
THRESHOLD = 0.45    #model training threshold for classification probability
EPSILON = 1e-16     #variable to force bounded solution for log trransforms


# INPUT_FILE = 'gs://261_projectdata/261project_data/df.parquet'
INPUT_FILE = 'data/df.parquet'

Assign column features and parquet files

In [12]:
my_cols = ['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7',
           'n8', 'n9', 'n10', 'n11', 'n12',
           'n13','cat14', 'cat15', 'cat16', 'cat17',
           'cat18', 'cat19', 'cat20', 'cat21', 'cat22',
           'cat23', 'cat24', 'cat25', 'cat26', 'cat27',
           'cat28', 'cat29', 'cat30', 'cat31', 'cat32',
           'cat33', 'cat34', 'cat35', 'cat36', 'cat37',
           'cat38', 'cat39']

df_pq = spark.read.parquet(INPUT_FILE)

In [13]:
print(f'Creating Hashed feature with {NUM_FEATURES} number of vectors ...')
start = time.time()

hasher = FeatureHasher(inputCols=my_cols, outputCol="features", numFeatures = NUM_FEATURES)

hash_transformed = hasher.transform(df_pq).select('y', 'features')
print(f'Hashed feature vector completed ...in {time.time() - start}')

df_pq.unpersist()

hash_transformed.show(n=5, truncate=True)
#create train-test split
print('Creating train-test split..')
train_y_all, test_y_all = hash_transformed.randomSplit([0.8, 0.2], seed=10)
hash_transformed.unpersist()
#change the name of the dependent variable column
print('Creating test set..')
test = test_y_all.selectExpr("y as labels", "features")
test.show(n=5)
#force a binary classification 0 or 1 on the training data and save as column 'labels'
print('Creating train set..')
train_y = train_y_all.withColumn('labels', when(col('y') > 0.0, lit(1)).otherwise(lit(0)))
train_y_all.unpersist()

train = train_y.drop('y')
train_y.unpersist()

train.show(n=10)


#Get the percentage of values in the positive class, to set the baseline log loss
mean_prob = train.select(mean(col('labels'))).collect()[0]['avg(labels)']
print(f'Positive class in the training data = {mean_prob * 100} %')

#assign mean probability as the baseline probability estimate
train_prob = train.withColumn('base_prob', lit(mean_prob))
train.unpersist()

# calculate baseilne logloss  
train_ll = train_prob.withColumn('logloss', when(col('labels') == 1.0, - log(col('base_prob') + EPSILON))\
                                       .otherwise( - log(1.0 - col('base_prob') + EPSILON)))
train_prob.unpersist()

mean_ll = train_ll.select(mean(col('logloss'))).collect()[0]['avg(logloss)']
print(f'Baseline Log loss for the training data = {mean_ll}')
#specify the model and hyperparameters
lr = LogisticRegression(featuresCol = 'features',
                        labelCol='labels', 
                        maxIter = MAX_ITER, 
                        standardization = False,
                        elasticNetParam = 0.0, 
                        threshold = THRESHOLD
                        )

start = time.time()
print('Training model...')
lr_model = lr.fit(train_ll)
time_taken = time.time() - start
print(f'Training completed ...in {time_taken}')

def sigmoid(rawPrediction):
    #apply affine transformation
    return 1 / (1 + math.exp(- rawPrediction))

def extract_from_vector(vec, i):
    """ Input: VectorUDT data type
        Output: float type at index (i)
    """
    try:
        return float(vec[i])
    except ValueError:
        return None

#register UDFs      
get_probability = udf(sigmoid, DoubleType())
get_val = udf(extract_from_vector, DoubleType())

#Get predictions on test data
result = lr_model.transform(test)

#Apply affine transformation to the linear regression output
result2 = result.withColumn('calc_prob', get_probability(get_val("rawPrediction", lit(1))))
# calculate log loss
result3 = result2.withColumn('logloss', when(col('labels') > 0.0, - log(col('calc_prob') + EPSILON))\
                                       .otherwise( - log(1.0 - col('calc_prob') + EPSILON)))


test_mean_ll = result3.select(mean(col('logloss'))).collect()[0]['avg(logloss)']
print(f'Baseline Log loss for the training data = {mean_ll}')
print(f'MODEL Log loss for the training data = {test_mean_ll}')

analyze = result3.select('labels', 'prediction')
analyze = analyze.withColumn("prediction", analyze["prediction"].cast(IntegerType()))
analyze = analyze.withColumn("labels", analyze["labels"].cast(IntegerType()))

#Create SQL queries table
sqlContext.registerDataFrameAsTable(analyze, "results")
#Get metrics for precision recall calculations
TP = spark.sql("""SELECT COUNT(prediction)
                    FROM results
                    WHERE labels = 1 AND prediction = 1
                    """).collect()[0][0]

TN = spark.sql("""SELECT COUNT(prediction)
                    FROM results
                    WHERE labels = 0 AND prediction = 0
                    """).collect()[0][0]

FP = spark.sql("""SELECT COUNT(prediction)
                    FROM results
                    WHERE labels = 0 AND prediction = 1
                    """).collect()[0][0]

FN = spark.sql("""SELECT COUNT(prediction)
                    FROM results
                    WHERE labels = 1 AND prediction = 0
                    """).collect()[0][0]

P = spark.sql("""SELECT COUNT(labels)
                    FROM results
                    WHERE labels = 1
                    """).collect()[0][0]

N = spark.sql("""SELECT COUNT(labels)
                    FROM results
                    WHERE labels = 0
                    """).collect()[0][0]

print(f'Precision = {TP/(TP + FP) * 100} %')

print(f'Recall = {TP/(TP + FN) * 100} %')

Creating Hashed feature with 262144 number of vectors ...
Hashed feature vector completed ...in 0.08662009239196777
+---+--------------------+
|  y|            features|
+---+--------------------+
|  0|(262144,[3723,576...|
|  0|(262144,[3723,398...|
|  0|(262144,[3723,654...|
|  0|(262144,[3723,141...|
|  1|(262144,[5768,129...|
+---+--------------------+
only showing top 5 rows

Creating train-test split..
Creating test set..
+------+--------------------+
|labels|            features|
+------+--------------------+
|     0|(262144,[26,5929,...|
|     0|(262144,[41,3723,...|
|     0|(262144,[67,3723,...|
|     0|(262144,[76,3723,...|
|     0|(262144,[89,3723,...|
+------+--------------------+
only showing top 5 rows

Creating train set..
+--------------------+------+
|            features|labels|
+--------------------+------+
|(262144,[9,15026,...|     0|
|(262144,[13,3723,...|     0|
|(262144,[32,15944...|     0|
|(262144,[35,3723,...|     0|
|(262144,[35,3723,...|     0|
|(262144,[38